In [89]:
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report
import torch
import matplotlib.pyplot as plt
import os

In [90]:
dataset_name = "2D_Helm/"

#Configurpave to be dynamically adjusted
download_path = "../data/" #In the .gitignore list an

#n the rest of the code.
path_to_datasets = download_path + "/" + dataset_name 

In [91]:
# This cell now makes use of the downloadfolder for the datasets.
df_train= pd.read_csv(path_to_datasets + "/" + 'helm_train.csv')
df_test=pd.read_csv(path_to_datasets + "/" +  'helm_test.csv')
#print("Dataframes MITBIH correctly read into workspace")

#split target and value
train_target=df_train['p(x,y)']
test_target=df_test['p(x,y)']
train=df_train.drop('p(x,y)',axis=1)
test=df_test.drop('p(x,y)',axis=1)

In [92]:
df_train.sample(1)

,x,y,"p(x,y)"
20,0.6,0.4,0.2


In [93]:
#Switches to decide the dataset sampling method and which models should be run
class Config_Sampling:
    oversample = False #equals to B_SMOTE
    undersample = False
    sample_name = "UNDEFINED_SAMPLE"
    
Train_Simple_ANN = True #Trains the simple ANN

The following functions should normally be imported from the other script

In [94]:
def grad_x(p, len_x): 
    """Calculate the gradient of the input p in x-direction
       It assumes uniformly distributed nodes in a domain D=[0,1] x [0,1].
       Hence, len_x = len_y.
       Adjust coordinates and geometry if needed, or alternatively transform variables.
    """
    epsilon = 1e-8 #
    dX = 1.0/(len_x-1) #Spacing h is denoted with dX
    gradp_x=torch.zeros_like(p) + epsilon

    for i in range(1,len_x-1):
        for j in range(0,len_x):
            gradp_x[i*len_x+j] = (p[(i+1)*len_x+j] - p[(i-1)*len_x+j]) * 0.5/dX

    #One-sided gradient for @ x=0. It means i==0. 
    for j in range(0,len_x):
        gradp_x[j] = ( -p[2*len_x+j] +4*p[len_x+j] - 3*p[j]) * 0.5/dX


    #One-sided gradient for @ x=1. It means i==lenX-1. Second-order accurate
    for j in range(0,len_x):
        gradp_x[(len_x-1)*len_x+j] =  (3*p[(len_x-1)*len_x+j] - 4*p[(len_x-2)*len_x+j] + p[(len_x-3)*len_x+j]) * 0.5/dX

    return gradp_x

In [95]:
def grad_y(p, len_x): 
    """Calculate the gradient of the input p in y-direction
       It assumes uniformly distributed nodes in a domain D=[0,1] x [0,1].
       Hence, len_x = len_y.
       Adjust coordinates and geometry if needed, or alternatively transform variables.
    """
    epsilon = 1e-8 #
    dX = 1.0/(len_x-1) #Spacing h is denoted with dX
    gradp_y=torch.zeros_like(p) + epsilon

    for i in range(0,len_x):
        for j in range(1,len_x-1):
            gradp_y[i*len_x+j] = (p[i*len_x+j+1] - p[i*len_x+j-1]) * 0.5/dX

    # One-sided @ y=0. Corresponds j=0.
    for i in range(0,len_x):
            gradp_y[i*len_x] = (-p[i*len_x+2] + 4*p[i*len_x+1] - 3*p[i*len_x]) * 0.5/dX

    # One-sided @ y=1. Corresponds j=lenY-1.
    for i in range(0,len_x):
            gradp_y[(i+1)*len_x-1] = (-p[(i+1)*len_x-3] + 4*p[(i+1)*len_x-2] - 3*p[(i+1)*len_x-1]) * 0.5/dX
    
    return gradp_y

## **Simple Artificial Neural Network**
ANN without convolutional layers. Only Dense layers are used. No Pooling, Flattening or Dropping out. Base model for later comparison.

In [96]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

Implement Torch Dataset object

In [97]:
class ECG_Dataset(Dataset):
    def __init__(self, csv_file, transform=None, target_transform=None):
        self.dataframe = csv_file.values
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.dataframe)
        #return self.dataframe.shape[0] # Alternative notation

    def __getitem__(self, idx):
        inputs = torch.tensor(self.dataframe[idx,:-1], requires_grad=True).to(torch.float32)
        label = torch.tensor(self.dataframe[idx,-1]).to(torch.float32)

        return inputs, label

Custom function for preprocessing (to elaborate later, currently just returns the input itself)

In [98]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


def preprocess(x):
    return x * torch.Tensor([1.0])

In [99]:
# Define the ANN model
class SimpleANN(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc0 = nn.Sequential(Lambda(preprocess))
        self.fc1 = nn.Linear(input_size, 24)
        self.fc2 = nn.Linear(24, 48)  
        self.fc3 = nn.Linear(48, 24) 
        self.fc4 = nn.Linear(24, 12)  
        self.fc5 = nn.Linear(12, output_size)  # Hidden to output layer
        self.fc6 = nn.Linear(6, output_size)  # Hidden to output layer
        self.relu = nn.LeakyReLU(negative_slope=0.001)    # Activation function
        self.dropout = nn.Dropout(p=0.1)
        self.sigmoid = nn.Sigmoid() 

    def forward(self, x):
        #x = self.fc0(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        return x

In [100]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [101]:
kappa = 0 * torch.pi

In [102]:
lenX = int(np.sqrt(len(df_train)))
lenY = lenX
batch_size = 64

In [103]:
def target_func(x,y, kappa): 
    pres = x*x - y*y
    return pres

In [104]:
class DirichletBC(nn.Module):
    def __init__(self):
        super(DirichletBC, self).__init__()

    def forward(self, inputs, target_func, kappa):
        """
        Impose Dirichlet BC on the boundary.
        Args:
        Returns:
            torch.Tensor: Computed loss (scalar).
        """
        DirichletValues = []
        DirichletIndex = []

        for i, (x,y) in enumerate(inputs):
            if x==0 or x==1 or y==0 or y==1:
                DirichletValues.append(target_func(inputs[i,0], inputs[i,1], kappa))
                DirichletIndex.append(i)
        
        DirichletValues = torch.tensor(DirichletValues,requires_grad=True)
        DirichletIndex = torch.tensor(DirichletIndex,requires_grad=False)
                   
        return DirichletIndex.unsqueeze(1), DirichletValues.unsqueeze(1)

In [105]:
def get_data(train_ds, valid_ds, bs, shuffle):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=shuffle),
        DataLoader(valid_ds, batch_size=bs),
    )

In [106]:
train_ds = ECG_Dataset(df_train)
test_ds = ECG_Dataset(df_test)
train_dl, test_dl = get_data(train_ds, test_ds, batch_size, shuffle=False)

Apply Boundary Conditions

In [107]:
bc = DirichletBC()
for inputs, labels in train_dl:
    # Watch out here. This calculated assuming one batch size=dataset_len
        
    DirichletIndx, DirichletVals = bc(inputs,target_func, kappa)

In [108]:
def batch_loss_train(outputs, labels, inputs, kappa, loss_fn, optimizer):
    loss = loss_fn(outputs, labels, inputs, kappa)
    
    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()

    return loss.item()

In [109]:
def batch_loss_test(outputs, labels, loss_fn):
    loss = loss_fn(outputs, labels)    
    return loss.item()

In [110]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, mse, mape = 0, 0, 0
    epsilon = 1e-8

    # Evaluating the model with torch.no_grad()    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            print(pred.view(lenX,lenX))
            # test_loss += loss_fn(pred, y, X,kappa) / len(X)
            mse += torch.sum((y - pred) ** 2).item() 
            y = y + epsilon
            mape += torch.mean(torch.abs((y - pred) / y)) 

    
    print(f"Test set => Accuracy: {(mse/size):>0.4f}, Avg loss: {test_loss:>8f} \n")

In [117]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    optimizer.zero_grad()
    train_loss = 0.0
    #all_outputs = torch.empty(0, 1, requires_grad=True)
    
    for inputs, labels in dataloader:
        # forward pass. Better without shuffle to keep the coordinates sorted
        
        outputs = model(inputs)
        print(outputs.view(lenX,lenX))

        train_loss += batch_loss_train(outputs,labels,lenX,lenY,loss_fn, optimizer)
    
    print(f'Train loss: {train_loss}')
    

In [112]:
mse_loss = nn.MSELoss()

In [113]:
class LaplaceLoss(nn.Module):
    def __init__(self):
        super(LaplaceLoss, self).__init__()

    def forward(self, preds, targets, len_x, len_y):
        """
        Compute the Calculate the Laplace eq. for predictions.
        labels (targets) should satisfy zero anyway
        """
                            
        gradp_x = grad_x(preds,len_x)
        gradp_y = grad_y(preds,len_y)

        grad_p_xx = grad_x(gradp_x,len_x)
        grad_p_yy = grad_y(gradp_y,len_y)
        laplacian = grad_p_xx + grad_p_yy 

        # Dirichlet BC loss
        DBCpreds = preds[DirichletIndx.squeeze(1)].clone()

        DBCLoss = mse_loss(DBCpreds, DirichletVals)
        PDELoss = mse_loss(laplacian, torch.zeros_like(laplacian))

        loss = PDELoss.sum() + DBCLoss.sum()
        
        return loss


In [114]:
def fit(epochs, model, loss_fn, opt, train_dl, valid_dl):
    for t in range(epochs):  
        print(f"Epoch {t+1}   -------------------------------")
        train_loop(train_dl, model, loss_fn, optimizer)
        #test_loop(train_dl, model, loss_fn)


In [115]:
# Define the model
torch.set_printoptions(precision=6)

input_size = 2  # Number of input features
output_size = 1  # Output size (e.g., regression or binary classification)
model = SimpleANN(input_size, output_size)

# Define loss and optimizer
criterion =  LaplaceLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [132]:
fit(100, model, criterion, optimizer, train_dl, test_dl)

Epoch 1   -------------------------------
tensor([[ 0.077538, -0.117386, -0.341251, -0.605213, -0.941801, -0.940985],
        [ 0.064482, -0.045870, -0.226378, -0.388013, -0.665631, -0.664077],
        [ 0.107886,  0.070180, -0.072213, -0.197093, -0.443224, -0.444513],
        [ 0.319689,  0.272286,  0.124339, -0.001673, -0.238203, -0.239663],
        [ 0.690105,  0.558473,  0.367217,  0.201243, -0.051861, -0.050596],
        [ 1.105490,  0.875322,  0.629001,  0.363658,  0.080362,  0.081074]],
       grad_fn=<ViewBackward0>)
Train loss: 0.0562249980866909
Epoch 2   -------------------------------
tensor([[ 0.077567, -0.117326, -0.341148, -0.605074, -0.941894, -0.941309],
        [ 0.064719, -0.045573, -0.226109, -0.387855, -0.665773, -0.664369],
        [ 0.107895,  0.070374, -0.072160, -0.197009, -0.443282, -0.444697],
        [ 0.319248,  0.272232,  0.124204, -0.001475, -0.238375, -0.239956],
        [ 0.689753,  0.558182,  0.366960,  0.201777, -0.051878, -0.050715],
        [ 1.1055